In [1]:
import pandas as pd
import pyodbc

In [2]:
username = 'vigrose'
password = 'slytherin10946'

In [3]:
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [4]:
sql_query = \
    '''
    SELECT DISTINCT 
    A.PARTY_ID, 
    A.POST_CD_ID,
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY,
    C.PURPOSE_TYPE_DESC,
    C.PURPOSE_TYPE_ID,
    C.PURPOSE_USG_DESC
    FROM 
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE 
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    AND 
    A.THRU_DT IS NULL
    ORDER BY A.FROM_DT DESC
    '''

In [5]:
OTHER = pd.read_sql(con=AMAEDW, sql=sql_query)

In [16]:
other = OTHER[OTHER.PURPOSE_TYPE_ID.isin([269,317])==False]

In [11]:
amaia = pd.read_csv('Insurance_Select.csv')

In [12]:
amaia.columns

Index(['PARTY_ID', 'ME', 'POST_CD_ID', 'FROM_DT', 'ADDR_TYPE', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE_CD', 'ZIP', 'POST_CD_ID_PPMA',
       'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA', 'CITY_PPMA',
       'STATE_CD_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POST_CD_ID_POLO',
       'ADDR_1_POLO', 'ADDR_2_POLO', 'ADDR_3_POLO', 'CITY_POLO',
       'STATE_CD_POLO', 'ZIP_POLO', 'FROM_DT_POLO', 'POST_KEY', 'Match'],
      dtype='object')

In [18]:
merge_2 = pd.merge(amaia, other, on=['POST_CD_ID','PARTY_ID'])

In [20]:
merge_2.groupby('PURPOSE_USG_DESC').count()

,PARTY_ID,ME,POST_CD_ID,FROM_DT_x,ADDR_TYPE,ADDR_1_x,ADDR_2_x,ADDR_3_x,CITY_x,STATE_CD_x,...,ADDR_1_y,ADDR_2_y,ADDR_3_y,CITY_y,STATE_CD_y,ZIP_y,FROM_DT_y,POST_KEY_y,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID
PURPOSE_USG_DESC,,,,,,,,,,,,,,,,,,,,,
AMA Web Site,150417,150417,150417,150417,150417,149965,76859,14383,150417,150417,...,150417,140944,130103,150417,150417,150417,150417,150417,150417,150417
AMC Address.,5637,5637,5637,5637,5637,5637,2234,256,5637,5637,...,5637,5579,5576,5637,5637,5637,5637,5637,5637,5637
Advantage AM News,38,38,38,38,38,38,18,4,38,38,...,38,38,36,38,38,38,38,38,38,38
Advantage Archive Journal,50,50,50,50,50,50,24,4,50,50,...,50,49,46,50,50,50,50,50,50,50
Advantage Journal of the American Medical Association,96,96,96,96,96,96,48,8,96,96,...,96,94,87,96,96,96,96,96,96,96
Billing Address.,1814,1814,1814,1814,1814,1814,396,55,1814,1814,...,1814,1760,1759,1814,1814,1814,1814,1814,1814,1814
Insurance,41354,41354,41354,41354,41354,41354,7695,757,41354,41354,...,41354,7695,757,41354,41354,41354,41354,0,41354,41354
Membership Address.,85,85,85,85,85,85,30,4,85,85,...,85,84,83,85,85,85,85,85,85,85
Membership Shipping.,4749,4749,4749,4749,4749,4749,1577,264,4749,4749,...,4749,4718,4715,4749,4749,4749,4749,4749,4749,4749


In [21]:
len(amaia)

1254594

In [30]:
merge_2[merge_2.Match==False][['ADDR_1_x','ADDR_1_y','ADDR_1_POLO','ADDR_1_PPMA','PURPOSE_TYPE_DESC','PURPOSE_USG_DESC','Match']]

,ADDR_1_x,ADDR_1_y,ADDR_1_POLO,ADDR_1_PPMA,PURPOSE_TYPE_DESC,PURPOSE_USG_DESC,Match
2,4700 Waters Ave,4700 Waters Ave,171 Ashley Ave,171 Ashley Ave,DEA,No Usage,False
8,6150 W Layton Ave,6150 W Layton Ave,47111 Monroe St,5730 County Road 305,DEA,No Usage,False
15,8901 Carti Way,8901 Carti Way,7400 Fannin St,7400 Fannin St,DEA,No Usage,False
23,2100 Madison Ave,2100 Madison Ave,300 Carson St,Ste 218,DEA,No Usage,False
29,1734 Cedar Creek Rd,1734 Cedar Creek Rd,5555 Peachtree Dunwoody Rd,5555 Peachtree Dunwoody Rd,DEA,No Usage,False
...,...,...,...,...,...,...,...
1952048,72 Cambridge Rd,72 Cambridge Rd,153 W 11th St,153 W 11th St,License - New York,No Usage,False
1952675,1594 Pinetree Dr,1594 Pinetree Dr,6606 W Broad St,6606 W Broad St,DEA,No Usage,False
1952676,1594 Pinetree Dr,1594 Pinetree Dr,6606 W Broad St,6606 W Broad St,License - Pennsylvania,No Usage,False
1952834,6606 W Broad St,6606 W Broad St,1540 5th Ave,5 Circle Dr,License - Virginia,No Usage,False


In [31]:
merge_2[merge_2.Match==False].to_csv('Add_Usage.csv', index=False)

In [32]:
other

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT,POST_KEY,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID,PURPOSE_USG_DESC
0,2964331,10155771,1225 Gerard Ave,,,Bronx,NY,10452,2020-03-20 01:36:26.000000,3330964.0,DEA,401,No Usage
1,2358562,10367564,123 Hampshire Rd,,,Great Neck,NY,11023,2020-03-20 01:36:26.000000,1882851.0,DEA,401,No Usage
2,2363053,10581868,601 Elmwood Ave,Box 604,,Rochester,NY,14642,2020-03-20 01:36:26.000000,6397461.0,DEA,401,No Usage
3,2194784,10586250,111 E 56th St,Ste 311,,New York,NY,10022,2020-03-20 01:36:26.000000,6469328.0,DEA,401,No Usage
4,2061782,10594785,345 E 37th St,Rm 208,,New York,NY,10016,2020-03-20 01:36:26.000000,6605224.0,DEA,401,No Usage
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16321937,1843386,16350655,PO Box 2087,,,Anniston,AL,36202,2005-12-16 20:20:34.878569,715767.0,Office,318,Preferred Professional Mailing Address
16321944,1844757,9662436,PO Box 3149,,,Del Mar,CA,92014,2005-12-16 20:20:34.867515,715482.0,Office,318,Preferred Professional Mailing Address
16321946,1844754,16090735,115 W Clay St,,,Sylacauga,AL,35150,2005-12-16 20:20:34.863486,715391.0,Office,318,Preferred Professional Mailing Address
16321950,1888585,16272437,PO Box 909,,,Satsuma,AL,36572,2005-12-16 20:20:34.854912,713779.0,Office,318,Preferred Professional Mailing Address
